In [3]:
import sqlite3
import sys
import pymysql
pymysql.install_as_MySQLdb()
import os
import re
from os import path


tables = ['DetectorGeo','PmtBlr','ChannelGain','ChannelMapping','ChannelMask',
          'PmtNoiseRms','ChannelPosition',
          'PMTFEMapping', 'PMTFELowFrequencyNoise']


def create_table_sqlite(cursorSqlite, connSqlite, table_path, table):
    #cursorMySql.execute('show create table {}'.format(table))
    with open(table_path, 'r') as infile:
        lines = infile.read().split(";")
        for line in lines:
            sql  = line
            
            # show create table will include comments for each row
            # Example: `MinRun` int(11) NOT NULL COMMENT 'Minimum run number for which valid'
            # that is not compatible with sqlite3, so we remove it.
            sql = re.sub(r" COMMENT\s+\'.*\'", "", sql)

            # it will also include: ENGINE=MyISAM DEFAULT CHARSET=latin1
            # this is not compatible either, so we remove it.
            sql = re.sub(r"\s*ENGINE.*", "", sql)

            # some tables may have KEYs defined: KEY `ElecID` (`SensorID`)
            # this syntax is different, so we remove it too.
            # This happens for instance in table ChannelGain
            sql = re.sub(r",\s*\n\s*KEY.*[\n,]", "", sql)
            #print(sql)
            cursorSqlite.execute(sql)
            connSqlite.commit()

def copy_all_rows(connSqlite, cursorSqlite, cursorMySql, table):
    # Get all data
    cursorMySql.execute('SELECT * from {0}'.format(table))
    data = cursorMySql.fetchall()

    # Insert all rows
    fields = '?'
    try:
        nfields = len(data[0])
        fields += (nfields-1) * ',?'
        cursorSqlite.executemany('INSERT INTO {0} VALUES({1})'.format(table,fields),data)
        connSqlite.commit()
    except IndexError:
        print('Table ' +table+' is empty.')


def loadDB(dbname : str, fRemove=False):
    print("Cloning database {}".format(dbname))
    dbfile = path.join('localdb.'+dbname+'.sqlite3')
    
    if fRemove:
        try:
            os.remove(dbfile)
        except:
            pass

    connSqlite = sqlite3.connect(dbfile)
    cursorSqlite = connSqlite.cursor()

    for table in tables:
        print('{}/{}'.format(dbname, table)+'.sql')
        table_path = '{}/{}'.format(dbname, table)+'.sql'
        # Create table
        create_table_sqlite(cursorSqlite, connSqlite, table_path, table)

if __name__ == '__main__':
    dbname = ['GaP_ext']
    for name in dbname:
        loadDB(name)

Cloning database GaP_ext
GaP_ext/DetectorGeo.sql
GaP_ext/PmtBlr.sql
GaP_ext/ChannelGain.sql
GaP_ext/ChannelMapping.sql
GaP_ext/ChannelMask.sql
GaP_ext/PmtNoiseRms.sql
GaP_ext/ChannelPosition.sql
GaP_ext/PMTFEMapping.sql
GaP_ext/PMTFELowFrequencyNoise.sql


### Check values

In [1]:
import gres.database.load_db as db

In [2]:
db.DataPMT('gap_ext', 50)

,SensorID,ChannelID,PmtID,Active,X,Y,coeff_blr,coeff_c,adc_to_pes,noise_rms,Sigma
0,0,0,PMT,1,0.000,0.000,0,0,29.83,0,14.70
1,1,1,PMT,1,36.253,2.949,0,0,15.27,0,12.81
2,2,2,PMT,1,-15.573,-32.871,0,0,25.05,0,12.28
3,3,3,PMT,1,-20.680,29.922,0,0,22.21,0,-10.68
4,4,4,PMT,1,20.680,-29.922,0,0,25.46,0,15.43
5,5,5,PMT,1,-36.253,-2.949,0,0,34.77,0,18.18
6,6,6,PMT,1,15.573,32.871,0,0,31.02,0,19.00
7,7,7,PMT,1,0.000,0.000,0,0,1.00,0,0.00


In [16]:
12500*7

87500